<a href="https://colab.research.google.com/github/KonApp/Stable_Diffusion_Flashcard_Generator/blob/main/flashcard_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

In [ ]:
pipe = pipe.to("cuda")

In [ ]:
import torch

prompt = "Child brushing teeth, ink style"

generator = torch.Generator("cuda").manual_seed(1024)

image = pipe(prompt, num_inference_steps=50, generator=generator).images[0]

image

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
num_cols = 3
num_rows = 4

prompt = ["Help, illustration"] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt, num_inference_steps=50).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

In [ ]:
ch = input("which images would you like to save")
all_images[int(ch)].save("./astro.png")

In [ ]:
import json
import time
import os

!rm -r ./flashcards/
os.mkdir("./flashcards/")

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

num_cols = 3
num_rows = 3

with open('./flashcards.json', 'r') as fcc_file:
    fcc_data = json.load(fcc_file)

    for i in fcc_data:
      print(i['prompt'])
      prompt = [i['prompt']] *num_cols
      all_images = []
      for j in range(num_rows):
        images = pipe(prompt, num_inference_steps=50).images
        all_images.extend(images)
      grid = image_grid(all_images, rows=num_rows, cols=num_cols)
      grid
      ch = input("Select an image to save")
      all_images[int(ch)].save("./flashcards/"+i["name"]+".png")

In [ ]:
!zip ./flashcards.zip ./flashcards/